In [17]:
#import packages
import numpy as np
import pandas as pd
import regex as re

game_data = pd.read_csv('data/clean_data.csv')
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18272 entries, 0 to 18271
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  18272 non-null  int64  
 1   AppID                       18272 non-null  int64  
 2   Name                        18272 non-null  object 
 3   Release date                18272 non-null  object 
 4   Estimated owners            18272 non-null  object 
 5   Peak CCU                    18272 non-null  int64  
 6   Required age                18272 non-null  int64  
 7   Price                       18272 non-null  float64
 8   DLC count                   18272 non-null  int64  
 9   About the game              18244 non-null  object 
 10  Supported languages         18272 non-null  object 
 11  Full audio languages        18272 non-null  object 
 12  Reviews                     18272 non-null  int64  
 13  Windows                     182

Supported Audio Languages

In [18]:
#extract audio
copy_game_data = game_data.copy()
audio_data = pd.DataFrame()
audio_data.insert(0, 'AppID', game_data['AppID'])
audio_data.insert(1, 'Name', game_data['Name'])
index = 0

for row in copy_game_data.itertuples(index = True, name = 'Data'):
    temp = re.sub('[\[\]\'\s]', '', row[12])   #remove square brackets and apostrophes from string
    temp = temp.split(',')    #split string by ',' into a list 

    if temp[0] == '':
        temp.clear()
        temp.append('None')                              

    length = len(temp)

    if temp[0] == 'None':
        length -= 1

    audio_data.at[index, 'Number of full audio languages'] = length   #store num of audio lang for each game
    
    audio_data.at[index, 'Full audio languages'] = temp   #store list of languages

    index += 1

if (audio_data.at[0, 'AppID'] == 1469160):
    audio_data.at[0, 'Full audio languages'] = list(['None'])

audio_data.to_csv("data/Audio.csv", index=False)   #overwrite audio.csv

In [19]:
#find all audio languages 
all_audio = list()
all_audio.clear()
for row in audio_data.itertuples(index = True, name = 'Data'):
    for item in row[4]:
        if item in all_audio:
            continue
        if item == 'None':
            continue
        if len(item) == 1:
            continue
        else:
            all_audio.append(item)

print(all_audio)
print('Number of audio languages', len(all_audio))

['English', 'Japanese', 'German', 'French', 'Russian', 'Italian', 'Spanish-Spain', 'Portuguese-Brazil', 'TraditionalChinese', 'Spanish-LatinAmerica', 'Turkish', 'SimplifiedChinese', 'Korean', 'Hungarian', 'Czech', 'Danish', 'Dutch', 'Norwegian', 'Polish', 'Swedish', 'Ukrainian', 'Greek', 'Portuguese', 'Thai', 'Finnish', 'Arabic', 'Bulgarian', 'Romanian', 'Vietnamese', 'Portuguese-Portugal', 'Indonesian', 'Hindi']
Number of audio languages 32


In [20]:
audio_sep_data = pd.DataFrame()
audio_sep_data.insert(0, 'AppID', game_data['AppID'])
audio_sep_data.insert(1, 'Name', game_data['Name'])
audio_sep_data.insert(2, 'Price', game_data['Price'])

for language in all_audio:
    audio_sep_data[language] = 'F'

audio_sep_data['None'] = 'F'

audio_sep_data.to_csv("data/Audio_separated.csv", index=False)


In [21]:
index = 0

for row in audio_data.itertuples(index = True, name = 'Data'):
    
    for item in row[4]:
        audio_sep_data.at[index, item] = 'T'
    index += 1

audio_sep_data.to_csv("data/Audio_separated.csv", index=False)
print(audio_sep_data.head())

     AppID                       Name  Price English Japanese German French  \
0  1469160             Wartune Reborn   0.00       F        F      F      F   
1  1178150      MazM: Jekyll and Hyde  14.99       F        F      F      F   
2   320150  Deadlings: Rotten Edition   3.99       T        T      F      F   
3  1026420                     WARSAW  23.99       F        F      F      F   
4   485000             Cthulhu Realms   0.00       T        F      F      F   

  Russian Italian Spanish-Spain  ... Thai Finnish Arabic Bulgarian Romanian  \
0       F       F             F  ...    F       F      F         F        F   
1       F       F             F  ...    F       F      F         F        F   
2       F       F             F  ...    F       F      F         F        F   
3       F       F             F  ...    F       F      F         F        F   
4       F       F             F  ...    F       F      F         F        F   

  Vietnamese Portuguese-Portugal Indonesian Hindi 

In [ ]:
#For the boxplots
#replace the TRUE values with the price
audio_sep_price = pd.read_csv('data/Audio_separated.csv').copy()
x=0
print(game_data.shape[0])
while x != game_data.shape[0]:

    for row in audio_sep_price.itertuples(index = True, name = 'Data'):
        audio_sep_price.loc[x] = audio_sep_price.loc[x].replace(to_replace='T', value=row[3])
        audio_sep_price.loc[x] = audio_sep_price.loc[x].replace(to_replace='F', value=None)
    
        x+=1

        if (x == game_data.shape[0]):
            break


audio_sep_price.to_csv('data/Audio_separated2.csv', index=False)